# screenerCB

blah blah

In [10]:
#| default_exp screenerCB

In [30]:
#| export
from pw.core import *
from pw.crawler import *
from pw.helper import *

In [12]:
#| export
def screener_url(t): return f"https://www.screener.in/company/{t}/consolidated/"

In [13]:
#| export
class AnualReportCB(Callback):
    """
    Callback helping traveling all the links available in the same domain.
    """
    def __init__(self, tickets):
        self.order = 1
        self.links = {t : None for t in tickets}


    async def  after_visit(self, crawler, idx):
        
        pg = crawler.pages[idx]
        await pg.wait(10000)
        show_more_ele = await pg.find_ele('//button[@class="a show-more-button"]')
        for i in show_more_ele:
            await i.left_click()
        #await pg.pause()
        
        annual_reports = await pg.find_ele('//div[@class="documents annual-reports flex-column"]//child::a')

        links = {(await tag.get_text()).split("from")[0].strip() : await tag.get_attribute("href") 
                      for tag in annual_reports if await tag.get_attribute("href")}
        

        for i in list(self.links.keys()):
            if i in pg.url:
                break
        self.links[i] = links


In [25]:
t  = ["TATAMOTORS", "ASHOKLEY", "OLECTRA", "FORCEMOT", "SMLISUZU"]
url = [screener_url(i) for i in t]
C = Crawl(len(t), url, [AnualReportCB(t)])
await C.run(headless=False, stealth= True,devtools=True )

In [28]:
#| hide 
import pandas as pd

# Assuming your_dict is your dictionary
df = pd.DataFrame(C.cbs[0].links)
df.to_csv('company1.csv')
df.head()

,TATAMOTORS,ASHOKLEY,OLECTRA,FORCEMOT,SMLISUZU
Financial Year 2024,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2023,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2022,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2021,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...


In [20]:
class ConcallsCB(Callback):
    """
    Callback helping traveling all the links available in the same domain.
    """
    def __init__(self, tickets):
        self.order = 2
        self.trans_script = {t : None for t in tickets}
        
    async def  after_visit(self, crawler, idx):
        
        pg = crawler.pages[idx]
        """
        show_more_ele = await pg.find_ele('//button[@class="a show-more-button"]')
        for i in show_more_ele:
            await i.left_click()
        """
        
        n = len(await pg.find_ele('//div[@class="documents concalls flex-column"]//child::li'))
        dic = {}

        for i in range(1, n+1):
            label = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//div[1]')
            label = (await label[0].get_text()).strip()

            trans = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//a[contains(text(), "Transcript")]')

            if trans is None: # fetch ppt if Transcript is not presnet
                trans = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//a[contains(text(), "PPT")]') 
            #print(label, trans)

            dic[label] = await trans[0].get_attribute("href")

            print(dic)
        
        for i in list(self.trans_script.keys()):
            if i in pg.url:
                break
        self.trans_script[i] = dic

In [21]:
t  = ["TATAMOTORS", "ASHOKLEY"]
url = [screener_url(i) for i in t]
C = Crawl(2, url, [AnualReportCB(t), ConcallsCB(t)])
await C.run(headless=False, stealth= True,devtools=True )
#C.cbs[1].trans_script['TATAMOTORS']


Error find : @https://www.screener.in/company/TATAMOTORS/consolidated/  for //div[@class="documents concalls flex-column"]//child::li[1]//a[contains(text(), "Transcript")] :->  Element not found.  
{'Jan 2025': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=e9ded3b7-11c4-4572-a0ba-e7fa5dac0a16.pdf'}
{'Jan 2025': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=e9ded3b7-11c4-4572-a0ba-e7fa5dac0a16.pdf', 'Nov 2024': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=b24117dc-df34-49c5-b2d7-1b1aa135a1fc.pdf'}
{'Jan 2025': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=e9ded3b7-11c4-4572-a0ba-e7fa5dac0a16.pdf', 'Nov 2024': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=b24117dc-df34-49c5-b2d7-1b1aa135a1fc.pdf', 'Aug 2024': 'https://www.tatamotors.com/wp-content/uploads/2024/08/TML-Q1-FY25-Earnings-Call-Transcript.pdf'}
{'Jan 2025': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=e9ded3b7-11c4-4572-a0ba-e7fa5dac0a16.pdf', 'Nov 2024'

In [31]:
#| hide
import nbdev; nbdev.nbdev_export()